In [1]:
import pandas as pd
import torch 
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
device = torch.device("cuda:0")

data_train = torch.tensor(pd.read_csv('fashion-mnist_train.csv', sep=",").values)
x_train = data_train[:,1:]
x_train = x_train/255.0
y_train = data_train[:,:1]



data_test = torch.tensor(pd.read_csv('fashion-mnist_test.csv', sep=",").values)
x_test = data_test[:,1:]
x_test = x_test/255.0
y_test = data_test[:,:1]


y_train = torch.nn.functional.one_hot(y_train).view(-1,10).type(torch.float32)
y_test = torch.nn.functional.one_hot(y_test).view(-1,10).type(torch.float32)


x_train,y_train = x_train.to(device), y_train.to(device)
#Pegar todos os valores da coluna 1
#print(data_train[:,1])
#photo = torch.tensor(x_train[0,:].view(28,28))
#plt.imshow(photo, cmap=plt.cm.binary)
#plt.show()

In [2]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, 5, padding=2) 
        self.conv2 = nn.Conv2d(32, 64, 5, padding=2) 
        self.conv3 = nn.Conv2d(64, 128, 5, padding=2) 

        self._to_linear = None
        if self._to_linear is None:
            x = torch.randn(28,28).view(-1,1,28,28)
            #Função convs é pra descobrir o tamanho da saída da conv3
            self.convs(x)

        self.fc1 = nn.Linear(self._to_linear,512)
        self.fc2 = nn.Linear(512,10)

    def convs(self,x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2,2))
        x = F.max_pool2d(F.relu(self.conv2(x)), (2,2))
        x = F.max_pool2d(F.relu(self.conv3(x)), (2,2))

        if self._to_linear is None:
            self._to_linear = x[0].shape[0] * x[0].shape[1] * x[0].shape[2]
        return x
    
    def forward(self,x):
        x = self.convs(x)
        x = x.view(-1, self._to_linear)

        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        
        return F.softmax(x, dim=1)

In [3]:
net = Net()
net.to(device)

import torch.optim as optim
import numpy as np

optimizer = optim.Adam(net.parameters(), lr=0.001)
loss_function = nn.BCEWithLogitsLoss()
batch_X = x_train[0:100].view(-1,1,28,28)
batch_y = y_train[0:100]


batch_X, batch_y = batch_X.to(device), batch_y.to(device)

net.zero_grad()
outputs = net(batch_X)


In [12]:
def fwd_pass(X, y, train=False):

    if train:
        net.zero_grad()
    outputs = net(X)
    matches  = [torch.argmax(i)==torch.argmax(j) for i, j in zip(outputs, y)]
    acc = matches.count(True)/len(matches)
    loss = loss_function(outputs, y)

    if train:
        loss.backward()
        optimizer.step()

    return acc, loss

def test(size=32):
    #Definindo imagens de teste aleatórias
    random_start = np.random.randint(len(x_test)-size)
    X,y = x_test[random_start:random_start+size], y_test[random_start:random_start+size]
    print(x_test.shape[0])
    with torch.no_grad():
        val_acc, val_loss = fwd_pass(X.view(-1,1,28,28).to(device),y.to(device))
    return val_acc, val_loss

In [14]:
import time

MODEL_NAME = f"model-{int(time.time())}"
BATCH_SIZE = 100
EPOCHS = 15

with open("model.log", "a") as f:
    for epochs in range(EPOCHS):
        for i in tqdm(range(0, len(x_train), BATCH_SIZE)):
            #Aqui pegaremos as imagens de 100 em 100
            #Por exemplo: estaremos na imagem 100, o batch size pegará de 100 (i) até 200 (i+BATCH_SIZE); BATCH_S=100
            #O view é só para confirmarmos o número de features, número de canais, largura e comprimento
            batch_X = x_train[i:i+BATCH_SIZE].view(-1,1,28,28)
            batch_y = y_train[i:i+BATCH_SIZE]
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)
            acc, loss = fwd_pass(batch_X, batch_y, train=True)
            if i % 10 ==0:
                val_acc, val_loss = test(size=100)
                f.write(f"{MODEL_NAME}, {round(time.time(),3)}, {round(float(acc),2)}, {round(float(loss),4)},{round(float(val_acc),2)}, {round(float(val_loss),4)}\n")
        
        print(loss)

 [01:36<00:02,  6.25it/s]10000
10000
 97%|█████████▋| 584/600 [01:37<00:02,  6.17it/s]10000
10000
 98%|█████████▊| 586/600 [01:37<00:02,  6.10it/s]10000
10000
 98%|█████████▊| 588/600 [01:37<00:01,  6.23it/s]10000
10000
 98%|█████████▊| 590/600 [01:38<00:01,  6.04it/s]10000
10000
 99%|█████████▊| 592/600 [01:38<00:01,  6.12it/s]10000
10000
 99%|█████████▉| 594/600 [01:38<00:00,  6.19it/s]10000
10000
 99%|█████████▉| 596/600 [01:39<00:00,  6.13it/s]10000
10000
100%|█████████▉| 598/600 [01:39<00:00,  6.15it/s]10000
10000
  0%|          | 0/600 [00:00<?, ?it/s]10000
tensor(0.6627, device='cuda:0', grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
  0%|          | 1/600 [00:00<01:45,  5.66it/s]
10000
  0%|          | 3/600 [00:00<01:43,  5.78it/s]10000
10000
  1%|          | 5/600 [00:00<01:44,  5.70it/s]10000
10000
  1%|          | 7/600 [00:01<01:40,  5.92it/s]10000
10000
  2%|▏         | 9/600 [00:01<01:37,  6.06it/s]10000
10000
  2%|▏         | 11/600 [00:01<01:38,  5.95it/s]10000
10000


In [6]:
correct = 0
total = 0

with torch.no_grad():
    for i in tqdm(range(len(x_test))):
        real_class = torch.argmax(y_test[i])
        net_out = net(x_test[i].view(-1, 1, 28, 28).to(device))[0]  # returns a list, 
        predicted_class = torch.argmax(net_out)

        if predicted_class == real_class:
            correct += 1
        total += 1
print("Accuracy: ", round(correct/total, 3))

100%|██████████| 10000/10000 [00:26<00:00, 372.58it/s]Accuracy:  0.1



In [7]:
torch.cuda.empty_cache()

AttributeError: module 'torch.cuda' has no attribute 'clear_cache'